In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy
import json
import random
import tensorflow
import tflearn
import pickle

curses is not supported on this machine (please install/reinstall curses for an optimal experience)








In [3]:
stemm = LancasterStemmer()

with open("index.json") as files:
    data = json.load(files)

# try:
#     with open("data.pickle", "rb") as f:
#         words,labels,training,output = pickle.load(f) 
# except:

words = []
labels = []
docs_x = []
docs_y = []

for question in data["questions"]:
    for pattern in question["patterns"]:
#         token_word = nltk.wordpunct_tokenize(pattern)
        words.append(pattern)
        docs_x.append(pattern)
        docs_y.append(question["tag"])

    if question["tag"] not in labels:
        labels.append(question["tag"])

words = [stemm.stem(w.lower()) for w in words]
# docs_x = [stemm.stem(d.lower()) for d in docs_x]
# words = sorted(list(set(words)))

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    token_word = [stemm.stem(w.lower()) for w in doc]
    for w in words:
        if w in token_word:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    training.append(bag)
    output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)
# with open("data.pickle", "wb") as f:
#     pickle.dump((words,labels,training,output), f)

tensorflow.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
# model.load("model7.tflearn")

In [4]:
words

['hi',
 'hey',
 'hello',
 'morn',
 'afternoon',
 'ev',
 'night',
 'bot',
 'ai',
 'robot',
 'ticket',
 'apply',
 'movy',
 'movy',
 'ticket',
 'cinem',
 'cinema']

In [4]:
docs_x

['hi',
 'hey',
 'hello',
 'morning',
 'afternoon',
 'evening',
 'night',
 'bot',
 'AI',
 'robot',
 'ticket',
 'apply',
 'movie',
 'movies',
 'tickets',
 'cinema',
 'cinemas']

In [5]:
docs_y

['greeting',
 'greeting',
 'greeting',
 'wish',
 'wish',
 'wish',
 'wish',
 'chatbot',
 'chatbot',
 'chatbot',
 'about',
 'about',
 'about',
 'about',
 'about',
 'about',
 'about']

In [6]:
 token_word

['c', 'i', 'n', 'e', 'm', 'a', 's']

In [6]:
model.fit(training, output, n_epoch=1500, batch_size = 8 , show_metric=True)
model.save("model9.tflearn")

Training Step: 6299  | total loss: 1.31779 | time: 0.005s
| Adam | epoch: 2100 | loss: 1.31779 - acc: 0.4403 -- iter: 16/17
Training Step: 6300  | total loss: 1.28964 | time: 0.008s
| Adam | epoch: 2100 | loss: 1.28964 - acc: 0.4963 -- iter: 17/17
--
INFO:tensorflow:C:\Users\Rohit\Desktop\Hied Projects\Chanakya-webapp\Chatbot\model9.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
def user_bag_conversion(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.wordpunct_tokenize(s)
    s_words = [stemm.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
                
    return numpy.array(bag)

In [ ]:
def front():
    
    while True:
        user = input("You: ")
        if user.lower() == "quit":
            break
            
        results = model.predict([user_bag_conversion(user, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        print(tag)
        
        for tg in data['questions']:
            if tg['tag'] == tag:
                responses = tg['response']
                
        print(random.choice(responses))

print("How can i help you..?")        
front()

How can i help you..?
You: Hi
about
You can check out some of the movie ticket websites here
You: movies
about
You can check out some of the movie ticket websites here
